# Model Fine-Tuning with Hugging Face 🎯

Fine-tuning adapts a pretrained model to your specific task by continuing training on task-specific data. This notebook demonstrates fine-tuning BERT on the **GLUE MRPC** (Microsoft Research Paraphrase Corpus) dataset for paraphrase detection.

## What You'll Learn:
- Loading datasets from Hugging Face Hub
- Tokenizing data for transformer models
- Using DataCollator for efficient batching
- Configuring training with TrainingArguments
- Training with the Trainer API
- Evaluating and making predictions

---

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

2026-01-02 19:41:14.166394: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-02 19:41:14.359869: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-02 19:41:16.268132: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-02 19:41:16.268416: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not 

### Import Libraries

We need `transformers` for models/tokenizers, `datasets` for data loading, and `sklearn` for evaluation metrics.

### Load Dataset

In [2]:
dataset = load_dataset("glue", "mrpc")
dataset

README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

**GLUE MRPC Dataset**: Given two sentences, predict if they are paraphrases (semantically equivalent).
- **Label 1**: Paraphrase (same meaning)
- **Label 0**: Not a paraphrase (different meaning)

In [3]:
dataset['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Let's examine a single training example to understand the data structure:

In [ ]:
threshold = 5
count = 0
for record in dataset['train']:
    if record['label']==0:
        print(record)
        if count >= threshold:
            break
        count+=1

{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", 'label': 0, 'idx': 1}
{'sentence1': 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'sentence2': 'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .', 'label': 0, 'idx': 3}
{'sentence1': 'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .', 'sentence2': 'The tech-laced Nasdaq Composite .IXIC rallied 30.46 points , or 2.04 percent , to 1,520.15 .', 'label': 0, 'idx': 6}
{'sentence1': 'That compared with $ 35.18 million , or 24 cents per share , in the year-ago period .', 'sentence2': 'Earnings were affected by a non-recurring $ 8 million tax benefit in the year-ago period .', 'label': 0, 'idx': 8}
{'sentence1': 'Shares of

Let's look at some negative examples (non-paraphrases) to understand when sentences have different meanings:

In [5]:
dataset['train'].features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': ClassLabel(names=['not_equivalent', 'equivalent']),
 'idx': Value('int32')}

Check the dataset features (column types and label mappings):

### Tokenization

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

We tokenize **both sentences together** so the model can learn their relationship. The tokenizer:
- Adds [CLS] at the start
- Separates sentences with [SEP]
- Creates attention masks

Using `batched=True` processes multiple examples at once for speed.

In [7]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Notice the dataset now has additional columns: `input_ids`, `attention_mask`, and `token_type_ids`.

In [8]:
# Prepare datasets for PyTorch
train_dataset = tokenized_dataset["train"].shuffle(seed=42)
valid_dataset = tokenized_dataset["validation"]
test_dataset = tokenized_dataset["test"]

### Prepare Train/Validation/Test Splits

Shuffle training data for better generalization. Keep validation and test sets in original order.

In [9]:
# Data collator is used for dynamic padding per batch. For example batch 1 has texts of 
# size 10, 12 and 15. batch 2 has sizes 8, 6 and 9. Due to collator batch 1 will be padded to
# max size of 15 whereas batch 2 will be padded to a max size of 9. The other option is to apply
# global padding that will tax max length from the entire dataset and pad all other text to that max
# length which is not efficient

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [10]:
# Check if CUDA is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

### Check GPU Availability

Training is **much faster** on GPU. If CUDA is available, we'll use it!

### Train / Fine Tune the Model

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load BERT with a **classification head** (a new linear layer on top). The `num_labels=2` creates a binary classifier.

⚠️ You'll see a warning about random weights - this is expected! The classification head is randomly initialized and will be trained.

In [12]:
# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}

### Define Evaluation Metrics

We compute both **accuracy** and **F1 score**. F1 is especially useful for imbalanced datasets as it considers both precision and recall.

In [13]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

### Configure Training

Key parameters:
- **output_dir**: Where to save checkpoints
- **eval_strategy**: Evaluate after each epoch
- **per_device_train_batch_size**: Samples per batch (reduce if OOM)
- **num_train_epochs**: Training iterations over full dataset
- **weight_decay**: L2 regularization to prevent overfitting
- **load_best_model_at_end**: Keep the best checkpoint based on accuracy

In [14]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

### Create the Trainer

The Trainer brings everything together: model, training arguments, datasets, data collator, and metrics function.

In [15]:
# You may get an error if mlflow or dagshub are installed. To remove that error, you can uninstall 
# mlflow and dagshub and then restart the kernel
trainer.train()

2026/01/02 19:41:32 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/02 19:41:32 INFO mlflow.store.db.utils: Updating database tables
2026/01/02 19:41:32 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/02 19:41:32 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2026/01/02 19:41:32 INFO alembic.runtime.migration: Running 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.356082,0.865196,0.903678
2,No log,0.396245,0.860294,0.903879
3,0.403700,0.520720,0.850490,0.895369


TrainOutput(global_step=690, training_loss=0.33454308717147163, metrics={'train_runtime': 122.8349, 'train_samples_per_second': 89.584, 'train_steps_per_second': 5.617, 'total_flos': 428577075854640.0, 'train_loss': 0.33454308717147163, 'epoch': 3.0})

### 🚀 Start Training!

This will take several minutes. You'll see:
- Loss decreasing over time
- Accuracy/F1 improving each epoch
- Checkpoints being saved

**Note**: If you get MLflow errors, uninstall `mlflow` and `dagshub`, then restart the kernel.

### Model Evaluation

In [16]:
# Evaluate on test set
results = trainer.evaluate(test_dataset)
results

{'eval_loss': 0.4148944914340973,
 'eval_accuracy': 0.8226086956521739,
 'eval_f1': 0.8702290076335878,
 'eval_runtime': 4.8794,
 'eval_samples_per_second': 353.525,
 'eval_steps_per_second': 22.134,
 'epoch': 3.0}

Evaluate on the held-out test set to see how well the model generalizes to unseen data:

### Predictions

In [17]:
# Perform predictions on new sentences
def predict(sentences):
    inputs = tokenizer(sentences["sentence1"], sentences["sentence2"], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=1)
    return predictions.cpu().numpy()

Create a prediction function that:
1. Tokenizes input sentence pairs
2. Passes through the model
3. Returns the predicted class (0 or 1)

In [18]:
sample_sentences = {
    "sentence1": [
        "PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So",
        "The Nasdaq composite index increased 10.73 , or 0.7 percent , to 1,514.77"
    ],
    "sentence2": [
        "Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So", 
        "The Nasdaq Composite index, full of technology stocks, was lately up around 18 points"
    ]
}
predictions = predict(sample_sentences)
print("Predictions:", predictions)

Predictions: [1 0]


### Test on New Examples

Let's test with two sentence pairs:
1. Similar sentences about executives reporting structure → should be **1** (paraphrase)
2. Different Nasdaq statistics → should be **0** (not paraphrase)